---
title: Interactive Visualization Notebook of Key Results

---

# Interactive Visualization Notebook of Key Results

This notebook creates interactive visualizations for exploring groundwater recharge patterns, river networks, and subsurface characteristics in California's Central Valley.

## Setup and Imports

**Key Libraries:**

- **`xarray` & `rioxarray`**: Handle multi-dimensional raster data with geospatial awareness
- **`geopandas`**: Manage vector geospatial data (rivers, basins, points)
- **`ipyleaflet`**: Create interactive Leaflet.js maps in Jupyter
- **`ipywidgets`**: Build interactive UI controls (dropdowns, sliders)
- **`numpy`**: Numerical operations
- **`pandas`**: Tabular data manipulation
- **`shapely`**: Geometric operations

In [1]:
# -------------------------------
# Imports
# -------------------------------
import xarray as xr
import geopandas as gpd
from ipyleaflet import (
    ImageOverlay,
    GeoJSON,
    TileLayer,
)
from utils import leaflet_bounds, scalar_to_base64_image, find_intersections, combine_rivers_gdf, format_valley_name, DualMapController

In [2]:
# -------------------------------
# Data Paths
# -------------------------------
export_dir = "./data_export"

# Vector data (geospatial boundaries and features)
vector_rivers = f"{export_dir}/rivers.geojson"
vector_subbasin = f"{export_dir}/subbasin.geojson"
vector_brackish = f"{export_dir}/brackish.geojson"

# Raster/gridded data
metric_zarr_path = f"{export_dir}/consolidated_metric_output.zarr"

# -------------------------------
# Map Configuration
# -------------------------------

center = [37.66335291403956, -120.69523554193438]  # Central Valley center
zoom = 7  # Regional zoom level
basemap = None  # Use custom tile layers instead of default basemap
map_width = "500px"
map_height = "800px"

## Configuration

Define file paths, coordinate system, and visualization parameters.

**Directory Structure:**
- `data_download/`: Raw data files (shapefiles, CSV, rasters)
- `data_intermediate/`: Processed/consolidated datasets (Zarr)

**Coordinate Reference System:**
- **EPSG:4326** (WGS84 lat/lon) - Standard for web mapping
- All data will be reprojected to this CRS for consistency

**Map Settings:**
- Center coordinates: Central Valley, California
- Zoom level: 7 (regional view)
- Map dimensions: 500×800px

In [3]:
# -------------------------------
# Load Vector Layers
# -------------------------------
brackish = gpd.read_file(vector_brackish)
rivers = gpd.read_file(vector_rivers)
# Combine river segments that share the same name
rivers = combine_rivers_gdf(rivers)
# TODO: QC this - some rivers have gaps and potential duplicate names

subbasins = gpd.read_file(vector_subbasin)
# Apply formatting to subbasin names
subbasins['Basin_Su_1'] = subbasins['Basin_Su_1'].apply(format_valley_name)

# Find points where rivers exit the basin boundary
river_intersections = find_intersections(rivers, subbasins)

# -------------------------------
# Load Raster Data (Consolidated Metrics)
# -------------------------------
ds = xr.open_zarr(metric_zarr_path)
ds = ds.drop_vars('spatial_ref')

## Data Loading and Preprocessing

Load all datasets and prepare them for visualization by:
1. Reprojecting to common CRS (EPSG:4326)
2. Processing geometries for performance
3. Computing derived features

### Processing Steps

**Vector Data (Rivers & Basins):**
- Load GeoJSON files with `geopandas`
- Dissolve river fragments with same name into single geometries
- Simplify geometries (~100m tolerance) for faster rendering
- Find river-basin boundary intersections (exit/inflow points)

**Point Data (Resistivity):**
- Load CSV with UTM coordinates
- Convert to GeoDataFrame with proper CRS (EPSG:3310 → 4326)
- Subsample to 10,000 points for performance

**Raster Data (Metrics):**
- Open Zarr dataset with `xarray`
- Reorder and sort dimensions for proper display
- Reproject from CA Albers (EPSG:3310) to WGS84 (EPSG:4326)

**Performance Optimizations:**
- Geometry simplification reduces vertex count
- Point subsampling reduces render time
- Lazy loading with Zarr for large datasets

In [10]:
# -------------------------------
# Basemap Tile Layers
# -------------------------------
# Ocean basemap with bathymetry from ESRI ArcGIS Online
l_ocean = TileLayer(
    url="https://server.arcgisonline.com/ArcGIS/rest/services/Ocean/World_Ocean_Base/MapServer/tile/{z}/{y}/{x}",
    name="Ocean/Water",
    opacity=1.0,
)

# Hillshade elevation for terrain context
l_elevation = TileLayer(
    url="https://server.arcgisonline.com/ArcGIS/rest/services/Elevation/World_Hillshade/MapServer/tile/{z}/{y}/{x}",
    name="Hillshade",
    opacity=1.0,
)

# -------------------------------
# Vector Layers (Lines & Polygons)
# -------------------------------
l_rivers = GeoJSON(
    data=rivers.__geo_interface__,
    style={"color": "blue", "weight": 1, "opacity": 0.7},
    name="Rivers",
)

l_subbasins = GeoJSON(
    data=subbasins.__geo_interface__,
    style={"color": "black", "weight": 1, "fill": False},
    name="Subbasins",
)

l_brackish = GeoJSON(
    data=brackish.__geo_interface__,
    style={"color": "black", "weight": 1, "fillColor": "gray", "fillOpacity": 1.0,},
    name="Brackish Areas"
)

# River exit/inflow points at basin boundary
intersections_layer = GeoJSON(
    data=river_intersections.__geo_interface__,
    point_style={
        "radius": 6,
        "color": "orange",
        "fillColor": "orange",
        "fillOpacity": 0.5,
        "width": 2
    },
    name="River Exits",
)

## Layer Creation

Create all map layers for interactive visualization. Each layer can be toggled on/off independently.

### Layer Types

**Tile Layers (Basemaps):**
- `ipyleaflet.TileLayer` fetches map tiles from remote servers
- Ocean layer: Shows water features and bathymetry
- Hillshade layer: Provides terrain context with shaded relief

**Vector Layers:**
- `ipyleaflet.GeoJSON` renders vector features client-side
- Rivers: Blue lines showing drainage network
- Subbasins: Black outlines defining hydrologic boundaries
- Intersections: Orange markers at river exit points

**Point Visualization:**
- **GeoJSON points**: Individual markers (very small radius for dense data)
- **Heatmap**: Density visualization using kernel density estimation
  - `radius`: Influence distance of each point (pixels)
  - `blur`: Smoothing factor
  - `gradient`: Color scale from low to high density

**Use Cases:**
- Points: Precise locations
- Heatmaps: Spatial patterns and sampling density

In [ ]:
#| label: fig:multi-maps

# -------------------------------
# Dual Map Controller Class
# -------------------------------

import geopandas as gpd

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import numpy as np

colors = [(0, 0.5, 0),    # Green
          (1, 1, 1)]    # White
positions = [0, 1]
green_r_map = LinearSegmentedColormap.from_list("green_white", list(zip(positions, colors)))

colors = [(1, 1, 1),    # Green
          (0, 0.5, 0)]    # White
positions = [0, 1]
green_map = LinearSegmentedColormap.from_list("green_white", list(zip(positions, colors)))

# Extract RGB values from the predefined RdBu colormap
cmap = plt.get_cmap('RdBu')

# Create a list of colors by sampling the colormap
colors = [cmap(i) for i in range(cmap.N)]

# Create the custom colormap
custom_rdbu = LinearSegmentedColormap.from_list("custom_rdbu", np.array(colors)[20:-20][::-1])

param_dict = {
    'fraction_coarse': {
        'vmin':0, 'vmax':100, 'short_label': 'FCD' , 'label': 'Fraction of coarse-dominated sediment (%)', 'cmap':custom_rdbu, 'log_scale':False
    },
    'path_length_norm': {
        'vmin':1, 'vmax':1000,  'short_label': 'Normalized distance', 'label': 'Normalized path length', 'cmap':green_r_map, 'log_scale':True
    },
    'path_to_no_flow': {
        'vmin':0, 'vmax':20, 'short_label': 'Depth to no flow','label': 'Depth to shallowest no-flow unit\n or base surface (m)', 'cmap':green_map, 'log_scale':False
    },
}


# -------------------------------
# Dual Map Visualization
# -------------------------------

# Initialize controller
controller = DualMapController(
    width=map_width,
    height=map_height,
    center=center,
    zoom=zoom,
    subbasins=subbasins,
    subbasin_column="Basin_Su_1",
    rivers_gdf=rivers,
    river_intersections_gdf=river_intersections,
    param_dict=param_dict,
)

# -------------------------------
# Create Raster Overlays
# -------------------------------
# Left map: Fraction Coarse Dominated (static reference)
fcd_params = param_dict['fraction_coarse']
fcd_ave = ds["fraction_coarse"]
fcd_layer = ImageOverlay(
    url=scalar_to_base64_image(
        fcd_ave, 
        cmap=fcd_params['cmap'], 
        vmin=fcd_params['vmin'], 
        vmax=fcd_params['vmax'],
        log_scale=fcd_params['log_scale'],
    ),
    bounds=leaflet_bounds(fcd_ave),
    opacity=1.0,
    name="Raster Dataset",  # Generic name for layer control
)

# Right map: Dynamic metric overlay (switchable)
current_dataset_name = "path_length_norm"
current_params = param_dict[current_dataset_name]
da_scalar = ds[current_dataset_name].sel(threshold=20).load()
scalar_overlay = ImageOverlay(
    url=scalar_to_base64_image(
        da_scalar, 
        cmap=current_params['cmap'],
        vmin=current_params['vmin'],
        vmax=current_params['vmax'],
        log_scale=current_params['log_scale'],
    ),
    bounds=leaflet_bounds(da_scalar),
    opacity=1.0,
    name="Raster Dataset",  # Generic name for layer control
)

# -------------------------------
# Assemble and Display Maps
# -------------------------------
controller.add_base_layers(l_rivers, l_subbasins, l_brackish, fcd_layer, scalar_overlay)
controller.add_controls()  # Must be called AFTER adding base layers
controller.add_river_intersections(intersections_layer)
controller.create_dataset_selector(ds, current_dataset_name)
controller.create_subbasin_selector(center=center, zoom=zoom)
controller.m1.add_layer()
controller.display()

NameError: name 'l_hillshade' is not defined

## Figures 2-7: Dual Synchronized Maps

Side-by-side interactive maps with synchronized views for comparative analysis.

### Architecture

**DualMapController Class:**
A custom controller that manages two linked maps with shared navigation and independent data layers.

**Key Features:**
1. **View Synchronization**: Zoom and pan are linked via `ipywidgets.jslink()`
2. **Independent Layers**: Each map displays different data for comparison
3. **Interactive Highlighting**: Click features to highlight across both maps
4. **Dynamic Data Updates**: Switch datasets and thresholds in real-time

### Map Layout

**Left Map (m1):**
- Fraction Coarse Dominated (FCD) raster overlay
- Subbasin selector with zoom-to-region
- River exit point markers (clickable)
- Shows static reference data

**Right Map (m2):**
- Switchable metric datasets (path length, recharge, etc.)
- Threshold slider for FCD filtering
- Same base layers and markers
- Shows dynamic analysis results

### Interactive Workflows

**Subbasin Selection:**
1. Choose from dropdown → map zooms to bounds
2. Subbasin outline highlighted with orange dashed border
3. Highlight persists until new selection
4. "All Regions" returns to default view

**River Selection:**
1. Click orange marker at river exit point
2. Full river geometry highlights in bright blue on both maps
3. Popup shows river name
4. Multiple river highlights overlay (compare drainage patterns)

**Dataset Exploration:**
1. Select metric from dropdown (right map only)
2. Adjust FCD threshold with slider
3. Right map updates raster overlay
4. Left map unchanged for reference comparison

### Technical Implementation

**Synchronization:**
```python
jslink((map1, 'center'), (map2, 'center'))
jslink((map1, 'zoom'), (map2, 'zoom'))
```
- JavaScript-level linking for smooth interaction
- No Python callback overhead
- Bi-directional synchronization

**Event Handlers:**
- `on_click` for feature selection
- `.observe()` for widget changes
- Layer management (add/remove) for highlights

**State Management:**
- Track current highlights to prevent duplicates
- Store popup references for cleanup
- Maintain active dataset/threshold selections

break

In [ ]:
# Backup TODO: Remove

from ipyleaflet import Map, GeoJSON, WidgetControl, LayersControl, basemaps, Popup
import ipywidgets as widgets
import geopandas as gpd
from IPython.display import display
from ipywidgets import jslink
from utils import create_colorbar_widget

param_dict = {
    'fraction_coarse': {
        'vmin':0, 'vmax':100, 'short_label': 'FCD' , 'label': 'Fraction of coarse-dominated sediment (%)', 'cmap':'RdBu_r', 'log_scale':False
    },
    'path_length_norm': {
        'vmin':1, 'vmax':1000,  'short_label': 'Normalized distance', 'label': 'Normalized path length', 'cmap':'Greens', 'log_scale':True
    },
    'path_to_no_flow': {
        'vmin':0, 'vmax':20, 'short_label': 'Depth to no flow','label': 'Depth to shallowest no-flow unit\n or base surface (m)', 'cmap':'Greens', 'log_scale':False
    },
}


class DualMapController:
    """
    Controller for synchronized dual maps with interactive controls.

    Manages two side-by-side ipyleaflet maps with:
    - Synchronized zoom and pan
    - Independent data layers
    - Interactive feature highlighting
    - Dynamic dataset selection
    - Dynamic colorbars
    """

    def __init__(
        self,
        width,
        height,
        center,
        zoom,
        subbasins,
        subbasin_column,
        rivers_gdf,
        river_intersections_gdf,
        param_dict,
    ):
        """Initialize dual map controller with configuration and data."""
        self.subbasins = subbasins
        self.subbasin_column = subbasin_column
        self.rivers_gdf = rivers_gdf
        self.river_intersections_gdf = river_intersections_gdf
        self.param_dict = param_dict

        # Create maps with explicit center and zoom
        layout = widgets.Layout(width=width, height=height)
        self.m1 = Map(
            center=center, zoom=zoom, basemap=basemaps.CartoDB.Positron, layout=layout
        )
        self.m2 = Map(
            center=center, zoom=zoom, basemap=basemaps.CartoDB.Positron, layout=layout
        )

        # Synchronize views using JavaScript linking
        jslink((self.m1, "center"), (self.m2, "center"))
        jslink((self.m1, "zoom"), (self.m2, "zoom"))

        # -------------------------------
        # Define Highlight Styles
        # -------------------------------
        # Subbasin highlight style (orange dashed border)
        self.highlight_style = {
            "color": "orange",
            "weight": 3,
            "opacity": 0.8,
            "fillColor": "orange",
            "fillOpacity": 0.1,
            "dashArray": "5, 5",
        }

        # River highlight style (bright blue glow effect)
        self.river_highlight_style = {
            "color": "#0099ff",
            "weight": 5,
            "opacity": 0.8,
            "lineCap": "round",
            "lineJoin": "round",
        }

        # -------------------------------
        # State Tracking
        # -------------------------------
        # Track highlight layers (created on demand)
        self.river_highlight_m1 = None
        self.river_highlight_m2 = None
        self.current_highlight = None  # Subbasin highlight

        # Track popup references for cleanup
        self.current_popup_m1 = None
        self.current_popup_m2 = None

        # Widget for displaying selected river name
        self.river_name_widget = widgets.HTML(value="")

        # Debug output widget
        self.debug_output = widgets.Output()
        
        # Colorbar widgets and controls (initialized later)
        self.colorbar_control_m1 = None
        self.colorbar_control_m2 = None
        
        # Subbasin dropdown widget (initialized later)
        self.subbasin_dropdown = None

    def add_base_layers(self, rivers, subbasins, fcd_layer, scalar_overlay):
        """Add base layers to both maps."""
        # Add to both maps
        self.m1.add_layer(rivers)
        self.m2.add_layer(rivers)
        self.m1.add_layer(subbasins)
        self.m2.add_layer(subbasins)

        # Map-specific overlays
        self.m1.add_layer(fcd_layer)
        self.m2.add_layer(scalar_overlay)

        # Store references for dynamic updates
        self.fcd_layer = fcd_layer
        self.scalar_overlay = scalar_overlay

    def _create_colorbar_m1(self, dataset_name):
        """Create/update colorbar for left map (FCD)."""
        params = self.param_dict[dataset_name]
        
        # Remove existing colorbar if present
        if self.colorbar_control_m1 is not None:
            self.m1.remove_control(self.colorbar_control_m1)
        
        # Create new colorbar
        colorbar_widget = create_colorbar_widget(
            vmin=params['vmin'],
            vmax=params['vmax'],
            cmap=params['cmap'],
            label=params['label'],
            width=80,
            height=300,
            bar_width=0.1,
        )
        
        # Add colorbar at topright (on top of dropdown)
        self.colorbar_control_m1 = WidgetControl(
            widget=colorbar_widget, 
            position='topright'
        )
        self.m1.add_control(self.colorbar_control_m1)
    
    def _create_colorbar_m2(self, dataset_name):
        """Create/update colorbar for right map."""
        params = self.param_dict[dataset_name]
        
        # Remove existing colorbar if present
        if self.colorbar_control_m2 is not None:
            self.m2.remove_control(self.colorbar_control_m2)
        
        # Create new colorbar
        colorbar_widget = create_colorbar_widget(
            vmin=params['vmin'],
            vmax=params['vmax'],
            cmap=params['cmap'],
            label=params['label'],
            width=80,
            height=300,
            bar_width=0.1,
        )
        
        # Add colorbar at topright (on top of dropdown)
        self.colorbar_control_m2 = WidgetControl(
            widget=colorbar_widget, 
            position='topright'
        )
        self.m2.add_control(self.colorbar_control_m2)

    def add_river_intersections(self, intersections_layer):
        """Add clickable river intersection markers to both maps."""
        self.intersections_layer = intersections_layer

        # Register click event handler
        self.intersections_layer.on_click(self._on_intersection_click)

        # Add to both maps
        self.m1.add_layer(self.intersections_layer)
        self.m2.add_layer(self.intersections_layer)

    def _on_intersection_click(self, feature, **kwargs):
        """
        Handle click on river intersection point.

        Highlights the full river geometry on both maps and displays
        a popup with the river name.
        """
        river_name = feature["properties"].get("river_name", "Unknown")
        coords = feature["geometry"]["coordinates"]

        # Update river name display
        self.river_name_widget.value = f"<b>Selected River:</b> {river_name}"

        # -------------------------------
        # Clean Up Previous Popups
        # -------------------------------
        if self.current_popup_m1 is not None:
            self.m1.remove_layer(self.current_popup_m1)
        if self.current_popup_m2 is not None:
            self.m2.remove_layer(self.current_popup_m2)

        # -------------------------------
        # Create New Popups
        # -------------------------------
        popup_html = widgets.HTML(f"<b>{river_name}</b>")
        self.current_popup_m1 = Popup(
            location=(coords[1], coords[0]),
            child=popup_html,
            close_button=True,
            auto_close=False,
            close_on_escape_key=True,
            name="River Info Popup",
        )

        popup_html2 = widgets.HTML(f"<b>{river_name}</b>")
        self.current_popup_m2 = Popup(
            location=(coords[1], coords[0]),
            child=popup_html2,
            close_button=True,
            auto_close=False,
            close_on_escape_key=True,
            name="River Info Popup",
        )

        self.m1.add_layer(self.current_popup_m1)
        self.m2.add_layer(self.current_popup_m2)

        # -------------------------------
        # Highlight River Geometry
        # -------------------------------
        matching_river = self.rivers_gdf[self.rivers_gdf["GNIS_Name"] == river_name]

        if not matching_river.empty:
            # Remove previous river highlights
            if self.river_highlight_m1 is not None:
                self.m1.remove_layer(self.river_highlight_m1)
            if self.river_highlight_m2 is not None:
                self.m2.remove_layer(self.river_highlight_m2)

            # Create new highlight layers
            self.river_highlight_m1 = GeoJSON(
                data=matching_river.__geo_interface__,
                style=self.river_highlight_style,
                name="River Highlight",
            )
            self.river_highlight_m2 = GeoJSON(
                data=matching_river.__geo_interface__,
                style=self.river_highlight_style,
                name="River Highlight",
            )

            self.m1.add_layer(self.river_highlight_m1)
            self.m2.add_layer(self.river_highlight_m2)

    def add_controls(self):
        """Add LayersControl to both maps (call AFTER adding base layers)."""
        self.m1.add_control(LayersControl(position="bottomleft", collapsed=False))
        self.m2.add_control(LayersControl(position="bottomleft", collapsed=False))

    def create_dataset_selector(self, ds, initial_dataset):
        """Create dataset selection controls for right map."""
        self.ds = ds
        self.current_dataset_name = initial_dataset
        self.da_scalar = ds[initial_dataset]

        # -------------------------------
        # Dataset Dropdown (use short_label)
        # -------------------------------
        # Create options mapping from short_label to dataset name
        dataset_options = {
            self.param_dict[v]['short_label']: v 
            for v in ds.data_vars if v not in ["fraction_coarse"]
        }
        
        dataset_dropdown = widgets.Dropdown(
            options=list(dataset_options.keys()),
            value=self.param_dict[initial_dataset]['short_label'],
            description="Dataset:",
            style={"description_width": "initial"},
        )
        
        # Store mapping for callback
        self.dataset_options = dataset_options
        dataset_dropdown.observe(self._on_dataset_change, names="value")

        # -------------------------------
        # Threshold Slider
        # -------------------------------
        self.slider = widgets.SelectionSlider(
            options=ds.fraction.values,
            value=20,
            description="FCD Threshold (%)",
            style={"description_width": "initial"},
        )
        self.slider.observe(self._on_threshold_change, names="value")

        # Add dropdown FIRST (will be at back)
        controls = widgets.VBox([dataset_dropdown, self.slider])
        widget_control = WidgetControl(widget=controls, position="topright")
        self.m2.add_control(widget_control)
        
        # Add colorbar AFTER (will be on top)
        self._create_colorbar_m2(initial_dataset)

    def _on_dataset_change(self, change):
        """Handle dataset selection change."""
        # Map from short_label back to dataset name
        short_label = change["new"]
        self.current_dataset_name = self.dataset_options[short_label]
        self.da_scalar = self.ds[self.current_dataset_name]
        
        # Update colorbar
        self._create_colorbar_m2(self.current_dataset_name)
        
        # Update overlay
        self._update_scalar_overlay()

    def _on_threshold_change(self, change):
        """Handle threshold slider change."""
        self._update_scalar_overlay()

    def _update_scalar_overlay(self):
        """Update right map overlay with new dataset/threshold."""
        threshold = self.slider.value
        params = self.param_dict[self.current_dataset_name]

        # Select data by threshold if dimension exists
        if "fraction" in self.da_scalar.dims:
            da_overlay = self.da_scalar.sel(fraction=threshold)
        else:
            da_overlay = self.da_scalar

        # Update image overlay with new data using param_dict values
        self.scalar_overlay.url = scalar_to_base64_image(
            da_overlay,
            cmap=params['cmap'],
            vmin=params['vmin'],
            vmax=params['vmax'],
        )

    def create_subbasin_selector(self, center, zoom):
        """Create subbasin selection dropdown for left map."""
        self.default_center = center
        self.default_zoom = zoom

        # Create dropdown options (no "All Regions")
        subbasin_names = self.subbasins[self.subbasin_column].tolist()
        self.subbasin_dropdown = widgets.Dropdown(
            options=subbasin_names,
            value=None,
            description="Subbasin:",
            style={"description_width": "initial"},
        )
        self.subbasin_dropdown.observe(self._on_subbasin_change, names="value")

        # Add dropdown FIRST (will be at back)
        controls_vbox = widgets.VBox([self.subbasin_dropdown, self.river_name_widget])
        control = WidgetControl(widget=controls_vbox, position="topright")
        self.m1.add_control(control)
        
        # Add colorbar AFTER (will be on top)
        self._create_colorbar_m1('fraction_coarse')

    def _on_subbasin_change(self, change):
        """Handle subbasin selection change - zooms and highlights region."""
        selected_name = change["new"]
        
        # If no selection, do nothing
        if selected_name is None:
            return

        # Remove previous highlight
        if self.current_highlight is not None:
            self.m1.remove_layer(self.current_highlight)
            self.m2.remove_layer(self.current_highlight)
            self.current_highlight = None

        # Find selected subbasin
        selected_subbasin = self.subbasins[
            self.subbasins[self.subbasin_column] == selected_name
        ]

        if selected_subbasin.empty:
            with self.debug_output:
                print("No matching subbasin found!")
            return

        # Zoom to subbasin bounds
        bounds = selected_subbasin.total_bounds
        self.m1.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])

        # Create highlight overlay
        self.current_highlight = GeoJSON(
            data=selected_subbasin.__geo_interface__,
            style=self.highlight_style,
            name="Region Highlight",
        )
        self.m1.add_layer(self.current_highlight)
        self.m2.add_layer(self.current_highlight)

    def _on_home_click(self, button):
        """Handle home button click - reset to default view."""
        # Remove any subbasin highlight
        if self.current_highlight is not None:
            self.m1.remove_layer(self.current_highlight)
            self.m2.remove_layer(self.current_highlight)
            self.current_highlight = None
        
        # Reset dropdown to no selection
        if self.subbasin_dropdown is not None:
            self.subbasin_dropdown.value = None
        
        # Reset to default view
        self.m1.center = self.default_center
        self.m1.zoom = self.default_zoom

    def create_home_button(self):
        """Create home button above the maps."""
        home_button = widgets.Button(
            description="Home",
            button_style="info",
            tooltip="Reset to default view",
            icon="home",
            layout=widgets.Layout(width="100px")
        )
        home_button.on_click(self._on_home_click)
        
        return home_button

    def display(self):
        """Display the dual map setup with home button above."""
        home_button = self.create_home_button()
        
        display(self.debug_output)
        display(home_button)
        display(widgets.HBox([self.m1, self.m2]))


# -------------------------------
# Dual Map Visualization
# -------------------------------

# Initialize controller
controller = DualMapController(
    width=map_width,
    height=map_height,
    center=center,
    zoom=zoom,
    subbasins=subbasins,
    subbasin_column="Basin_Su_1",
    rivers_gdf=rivers,
    river_intersections_gdf=river_intersections,
    param_dict=param_dict,
)

# -------------------------------
# Create Raster Overlays
# -------------------------------
# Left map: Fraction Coarse Dominated (static reference)
fcd_params = param_dict['fraction_coarse']
fcd_ave = ds["fraction_coarse"]
fcd_layer = ImageOverlay(
    url=scalar_to_base64_image(
        fcd_ave, 
        cmap=fcd_params['cmap'], 
        vmin=fcd_params['vmin'], 
        vmax=fcd_params['vmax']
    ),
    bounds=leaflet_bounds(fcd_ave),
    opacity=1.0,
    name="Raster Dataset",  # Generic name for layer control
)

# Right map: Dynamic metric overlay (switchable)
current_dataset_name = "path_length_norm"
current_params = param_dict[current_dataset_name]
da_scalar = ds[current_dataset_name].sel(threshold=20).load()
scalar_overlay = ImageOverlay(
    url=scalar_to_base64_image(
        da_scalar, 
        cmap=current_params['cmap'],
        vmin=current_params['vmin'],
        vmax=current_params['vmax']
    ),
    bounds=leaflet_bounds(da_scalar),
    opacity=1.0,
    name="Raster Dataset",  # Generic name for layer control
)

# -------------------------------
# Assemble and Display Maps
# -------------------------------
controller.add_base_layers(l_rivers, l_subbasins, fcd_layer, scalar_overlay)
controller.add_controls()  # Must be called AFTER adding base layers
controller.add_river_intersections(intersections_layer)
controller.create_dataset_selector(ds, current_dataset_name)
controller.create_subbasin_selector(center=center, zoom=zoom)
controller.display()

AttributeError: 'Dataset' object has no attribute 'fraction'